In [1]:
import os
import re
import json
import tokenizers
import transformers

import pandas as pd

from tqdm import tqdm
from pathlib import Path

In [2]:
def basicPreprocess(text):
    processed_text = text.lower()
    processed_text = re.sub(r'\W +', ' ', processed_text)
    return processed_text

In [3]:
complete_df = pd.read_csv("data/clean_df.csv")

In [4]:
data = complete_df.sample(frac = 1).sample(frac = 1)

In [5]:
data.dropna(inplace = True)

In [6]:
data = data["abstract"].apply(basicPreprocess).replace("\n"," ")

In [7]:
txt_files_dir = "text_splits"

In [8]:
i = 0
for row in tqdm(data.to_list()):
    file_name = os.path.join(txt_files_dir, str(i)+'.txt')
    f = open(file_name, 'w')
    f.write(row)
    f.close()
    i += 1

100%|██████████| 40152/40152 [00:06<00:00, 5982.47it/s]


## Initialize Tokenizer

In [9]:
paths = [str(x) for x in Path(txt_files_dir).glob("**/*.txt")]

tokenizer = tokenizers.ByteLevelBPETokenizer()

vocab_size = 5000

tokenizer.train(files = paths, vocab_size = vocab_size, min_frequency = 5, special_tokens = [
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

## Train Test Split

In [10]:
lm_data_dir = "lm_data"

In [11]:
train_split = 0.9
train_data_size = int(len(data)*train_split)

with open(os.path.join(lm_data_dir,'train.txt') , 'w') as f:
    for item in data[:train_data_size].tolist():
        f.write("%s\n" % item)

with open(os.path.join(lm_data_dir,'eval.txt') , 'w') as f:
    for item in data[train_data_size:].tolist():
        f.write("%s\n" % item)

## Model and Tokenizer Instantiation

In [12]:
tokenizer.save("models/COVID")

['models/COVID/vocab.json', 'models/COVID/merges.txt']

In [13]:
train_path = os.path.join(lm_data_dir,"train.txt")
eval_path = os.path.join(lm_data_dir,"eval.txt")

In [14]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = tokenizers.implementations.ByteLevelBPETokenizer(
    'models/COVID/vocab.json', 'models/COVID/merges.txt'
)

In [16]:
tokenizer._tokenizer.post_processor = tokenizers.processors.BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

In [17]:
config = transformers.RobertaConfig(
    vocab_size = 5000,
    max_position_embeddings = 514,
    num_attention_heads = 4,
    num_hidden_layers = 1,
    type_vocab_size = 1,
)

In [19]:
model = transformers.RobertaForMaskedLM(config)

In [21]:
tokenizer = transformers.RobertaTokenizerFast.from_pretrained("models/COVID")

In [22]:
dataset = transformers.LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = "lm_data/train.txt",
    block_size = 128,
)

In [23]:
data_collator = transformers.DataCollatorForLanguageModeling(
    tokenizer = tokenizer, mlm = True, mlm_probability = 0.15
)

In [24]:
training_args = transformers.TrainingArguments(
    output_dir = "models/COVID",
    overwrite_output_dir = True,
    num_train_epochs = 10,
    per_gpu_train_batch_size = 64,
    save_steps = 10_000,
    save_total_limit = 2,
)

trainer = transformers.Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = dataset,
    prediction_loss_only = True,
)

You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.


In [25]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


{"loss": 6.9621486053466795, "learning_rate": 5.752212389380531e-06, "epoch": 0.8849557522123894, "step": 500}




TrainOutput(global_step=565, training_loss=6.941924107813202)

In [26]:
trainer.save_model("models/COVID")

In [31]:
model = transformers.RobertaForMaskedLM.from_pretrained('models/COVID')

In [33]:
tokenizer = transformers.RobertaTokenizerFast.from_pretrained('models/COVID')

In [34]:
import torch

In [35]:
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens = True)).unsqueeze(0)

In [37]:
outputs = model(input_ids)

In [44]:
type(outputs[0])

torch.Tensor

In [40]:
model.num_parameters()

12512648